In [3]:
import os
from PIL import Image, ImageDraw
import importlib
import config


In [4]:
#reload config whenever there are changes
importlib.reload(config)

<module 'config' from 'c:\\01 Github Code\\Ober-Chorochromatic-Map-Digitiser\\Chorochromatic-Map-Digitiser\\config.py'>

In [5]:


# Parameters straight from config.py
tile_size = config.tile_size
original_image_path = config.original_image_path
log_file_path = config.log_file_path
reconstructed_image_path = config.reconstructed_image_path


# Matching classes to colours
class_colors = {
    'Class-0-water': (185, 228, 201),  
    'Class-1': (254, 232, 169),       
    'Class-2': (250, 200, 83),         
    'Class-3': (196, 119, 77),       
    'Class-4': (135, 75, 89),    
    'Class-5': (114, 56, 77)       
}

# Load the original image
original_image = Image.open(original_image_path)
img_width, img_height = original_image.size

# and... create a new blank image with the same size
reconstructed_image = Image.new('RGB', (img_width, img_height))

# Function to draw a tile of the desired size...
def draw_tile(draw, x, y, color, tile_size):
    box = [x, y, x + tile_size, y + tile_size]
    draw.rectangle(box, fill=color)

# Read the log file reated in 03-map-tiles-classifier-with-tflite and draw the tiles

with open(log_file_path, 'r') as log_file:
    draw = ImageDraw.Draw(reconstructed_image)
    for line in log_file:
        tile_info = line.strip().split(',')
        tile_path = tile_info[0]
        tile_class = tile_info[1]
        tile_name = os.path.basename(tile_path)
        
        # Extract the x, y coordinates from the tile name
        x, y = map(int, tile_name.replace('tile_', '').replace('.png', '').split('_'))
        
        # get the color
        color = class_colors[tile_class]
        
        # draw the tileeee
        draw_tile(draw, x, y, color, tile_size)

# save the  image

reconstructed_image.save(reconstructed_image_path)

print(f"Reconstructed image saved as {reconstructed_image_path}")


Reconstructed image saved as ./Reconstructed_WRAP_map.png


And now we just need to apply the world file of the original map and it can be brought into QGIS nice and easy
In QGIS, once vectorised, we will need to delete all the feautures of very small area to get rid of the rogue polygons.